In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [2]:
!pip install -U -q PyDrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='AI generated fake text detection.csv'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('AI generated fake text detection.csv')

In [4]:
import pandas as pd
import seaborn as sns

In [5]:
df = pd.read_csv('AI generated fake text detection.csv')
df

,Timestamp,Original text,ChatGPT Generated Text,Google Bard Generated Text
0,8/11/2023 0:55:39,"In this paper, we consider the transmission of...",This paper explores the transmission of confid...,"In this paper, we study the secure transmissio..."
1,8/11/2023 0:57:27,Wireless transmission frequently deals with ch...,Channels exhibiting multipath fading are a com...,Wireless transmission often encounters channel...
2,8/11/2023 0:58:39,A range of relatively simple and accurate stat...,Scientists have put forth a variety of statist...,Scientists have proposed a variety of relative...
3,8/11/2023 1:00:01,A statistical model was established in [1] whe...,"In reference [1], a statistical model was form...","In [1], a statistical model was developed that..."
4,8/11/2023 1:01:33,Barros and Rodrigues characterized secrecy cap...,"In their work [2], Barros and Rodrigues introd...","In [2], Barros and Rodrigues defined secrecy c..."
...,...,...,...,...
722,8/28/2023 10:42:24,Our experimental design includes several simul...,Our experimental design encompasses a range of...,Our experimental design includes several simul...
723,8/28/2023 10:44:07,"In Section 2, we briefly describe datasets and...","In Section 2, we provide a concise overview of...","In Section 2, we briefly describe the datasets..."
724,8/28/2023 10:45:18,"In Section 3, we describe the extensive experi...","In Section 3, we detail the comprehensive expe...","In Section 3, we describe the extensive experi..."
725,9/11/2023 21:30:57,The computerization of our society has substan...,The computerization of our society has greatly...,The computerization of our society has had a m...


**Creating a new dataset**

In [6]:
data = pd.DataFrame(columns=['Text','Label'])
data

,Text,Label


In [7]:
df = pd.DataFrame(df)

**Append orginal test to it with label 0**

In [8]:
for datas in df['Original text ']:
    data = data._append({'Text': datas,'Label': 0},ignore_index=True)

data

,Text,Label
0,"In this paper, we consider the transmission of...",0
1,Wireless transmission frequently deals with ch...,0
2,A range of relatively simple and accurate stat...,0
3,A statistical model was established in [1] whe...,0
4,Barros and Rodrigues characterized secrecy cap...,0
...,...,...
722,Our experimental design includes several simul...,0
723,"In Section 2, we briefly describe datasets and...",0
724,"In Section 3, we describe the extensive experi...",0
725,The computerization of our society has substan...,0


**Append AI generated data to it with label 1**

In [9]:
for datas in df['ChatGPT Generated Text']:
    data = data._append({'Text': datas,'Label': 1},ignore_index=True)

# for datas in df['Google Bard Generated Text']:
#     data = data._append({'Text': datas,'Label': 1},ignore_index=True)

In [10]:
data

,Text,Label
0,"In this paper, we consider the transmission of...",0
1,Wireless transmission frequently deals with ch...,0
2,A range of relatively simple and accurate stat...,0
3,A statistical model was established in [1] whe...,0
4,Barros and Rodrigues characterized secrecy cap...,0
...,...,...
1449,Our experimental design encompasses a range of...,1
1450,"In Section 2, we provide a concise overview of...",1
1451,"In Section 3, we detail the comprehensive expe...",1
1452,The computerization of our society has greatly...,1


In [11]:
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00


In [12]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from torch.utils.data import TensorDataset, DataLoader

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 labels for binary classification (0 for original, 1 for AI-generated)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame 'data' with 'Text' and 'Label' columns
X = data['Text'].tolist()
y = data['Label'].tolist()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# Tokenize the input data
max_seq_length = 128  # You can adjust this value as needed
train_inputs = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt", max_length=max_seq_length)
test_inputs = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt", max_length=max_seq_length)


In [15]:
# Convert labels to tensors
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)

# Create DataLoader for batching
batch_size = 32
train_data = TensorDataset(train_inputs["input_ids"], train_inputs["attention_mask"], train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size)

In [16]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-tune the model
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the specified device
model.to(device)

# ...
# Other parts of the code (loading data, training, and evaluation) remain the same


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ...

with torch.no_grad():
    test_inputs = {key: val.to(device) for key, val in test_inputs.items()}
    logits = model(**test_inputs).logits
    test_pred = logits.argmax(dim=1).cpu().numpy()
    accuracy = accuracy_score(y_test, test_pred)
    print("Accuracy: ", accuracy * 100)

    # Print confusion matrix and classification report
    print(confusion_matrix(y_test, test_pred))
    print("\n")
    print(classification_report(y_test, test_pred))


Accuracy:  80.75601374570446
[[129  20]
 [ 36 106]]


              precision    recall  f1-score   support

           0       0.78      0.87      0.82       149
           1       0.84      0.75      0.79       142

    accuracy                           0.81       291
   macro avg       0.81      0.81      0.81       291
weighted avg       0.81      0.81      0.81       291



In [19]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 labels for binary classification (0 for original, 1 for AI-generated)

# Function to predict AI-generated percentage
def predict_ai_generated_percentage(text_input):
    # Tokenize the input data
    max_seq_length = 128  # You can adjust this value as needed
    inputs = tokenizer(text_input, truncation=True, padding=True, return_tensors="pt", max_length=max_seq_length)

    # Move the model to the specified device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Get the model's prediction
    with torch.no_grad():
        logits = model(**inputs).logits
        probabilities = torch.softmax(logits, dim=1)[0]  # Get probabilities for the first sample
        ai_generated_percentage = probabilities[1].item() * 100  # Probability of being AI-generated
        return ai_generated_percentage

# Take user input for testing text
user_input = input("Enter the text for testing: ")

# Call the prediction function
ai_generated_percentage = predict_ai_generated_percentage(user_input)

print(f"The text is AI-generated with a probability of {ai_generated_percentage:.2f}%")
print(f"The text is Original with a probability of {100 - ai_generated_percentage:.2f}%")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter the text for testing: Scientists have put forth a variety of statistical models for fading channels, which are both relatively straightforward and precise. These models take into account the specific propagation environment and the communication scenario at hand.
The text is AI-generated with a probability of 39.18%
The text is Original with a probability of 60.82%
